YOLO Batch Inference Pipeline

This notebook executes inference on test dataset using the trained YOLO model. It performs batch prediction, collects detection results, and provides per-class analysis of model performance. The inference pipeline is designed for production evaluation and can process multiple images sequentially with configurable confidence thresholds.

Inference Workflow:
1. Load trained model from weights file
2. Configure inference parameters (confidence threshold 0.5, IoU threshold 0.45)
3. Execute batch prediction on test image collection
4. Extract and analyze detection results
5. Compute per-class detection statistics
6. Generate confidence score distributions
7. Export inference results for further analysis

In [ ]:
import torch
import mlflow
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

mlflow.set_tracking_uri('file:///mlruns')

DATA_DIR = Path('../data')
MODELS_DIR = Path('../models')
MODEL_PATH = MODELS_DIR / 'yolo_run' / 'weights' / 'best.pt'

print("YOLO INFERENCE PIPELINE")
print("=" * 50)
print(f"Loading model from: {MODEL_PATH}")

if MODEL_PATH.exists():
    model = YOLO(str(MODEL_PATH))
    print("Model loaded successfully")
else:
    print("Error: Model not found. Run training notebook first.")
    raise FileNotFoundError(str(MODEL_PATH))

In [ ]:
test_images_dir = DATA_DIR / 'images' / 'test'
test_images = list(test_images_dir.glob('*.jpg'))[:5]

results_list = []

for img_path in test_images:
    results = model.predict(
        source=str(img_path),
        conf=0.5,
        iou=0.45,
        verbose=False
    )
    results_list.append(results[0])
    
    detection_count = len(results[0].boxes) if results[0].boxes is not None else 0
    print(f"{img_path.name}: {detection_count} detections")

print(f"\nProcessed {len(test_images)} test images")
print("=" * 50)

In [ ]:
class_names = {0: 'person', 1: 'car', 2: 'dog'}

total_detections = {cls: 0 for cls in class_names.values()}

for result in results_list:
    if result.boxes is not None:
        for box in result.boxes:
            class_id = int(box.cls[0])
            confidence = float(box.conf[0])
            class_name = class_names[class_id]
            total_detections[class_name] += 1
            print(f"Detected {class_name} with confidence {confidence:.3f}")

print("\n" + "=" * 50)
print("Detection Summary:")
for cls_name, count in total_detections.items():
    print(f"  {cls_name}: {count} detections")

print("\nInference completed successfully")
print("Classes: person, car, dog")
print("Confidence threshold: 0.5")